# Jupyter notebook for CalLat gA project.
### Import libraries

In [39]:
import pandas as pd
pd.options.display.max_rows = 6
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import lsqfit
import gvar as gv
print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("mpl    version:", mpl.__version__)
print("lsqfit version:", lsqfit.__version__)
print("gvar   version:", gvar.__version__)

pandas version: 0.20.1
numpy  version: 1.12.1
mpl    version: 2.0.2
lsqfit version: 9.1.3
gvar   version: 8.3.2


### Define analysis parameters

In [40]:
switches = dict()
# Ensembles used in extrapolation
switches['ensembles'] = ['l1648f211b580m013m065m838','l2448f211b580m0064m0640m828','l3248f211b580m00235m0647m831',
                'l2464f211b600m0170m0509m635','l2464f211b600m0130m0509m635','l2464f211b600m0102m0509m635','l2464f211b600m00507m0507m628','l3264f211b600m00507m0507m628','l4064f211b600m00507m0507m628', 'l4864f211b600m00184m0507m628',
                'l3296f211b630m0074m037m440','l4896f211b630m00363m0363m430']

switches['ansatz'] = dict()
switches['ansatz']['type'] = 'Xpt' # Type of fit: 'Xpt' or 'taylor'
switches['ansatz']['truncation'] = 4 # Order to truncate series

### Import data

In [59]:
# import correlator bootstraps
gadf = pd.read_csv('./github_ga_v1.csv')
gadf

,ensemble,nbs,ga,epi,mpil
0,l1648f211b580m013m065m838,0,1.214927,0.249456,3.776239
1,l1648f211b580m013m065m838,1,1.210567,0.249160,3.781370
2,l1648f211b580m013m065m838,2,1.223266,0.249314,3.764355
...,...,...,...,...,...
60009,l4896f211b630m00363m0363m430,4998,1.261010,0.182535,4.694499
60010,l4896f211b630m00363m0363m430,4999,1.259121,0.182055,4.699100
60011,l4896f211b630m00363m0363m430,5000,1.252576,0.181791,4.698476


In [69]:
# import HISQ parameters
hqdf = pd.read_csv('./hisq_params.csv')
hqdf

,ensemble,aw0_mean,aw0_sdev,alfs
0,l1648f211b580m013m065m838,0.8804,0.0003,0.58801
1,l2448f211b580m0064m0640m828,0.8804,0.0003,0.58801
2,l3248f211b580m00235m0647m831,0.8804,0.0003,0.58801
...,...,...,...,...
9,l4864f211b600m00184m0507m628,0.7036,0.0005,0.53796
10,l3296f211b630m0074m037m440,0.5105,0.0003,0.43356
11,l4896f211b630m00363m0363m430,0.5105,0.0003,0.43356


### Format data for analysis

In [73]:
gar_list = []
epi_list = []
aw0_list = []
afs_list = []
mpl_list = []
for ens in switches['ensembles']:
    gar = gadf.query("ensemble=='%s'" %ens).sort_values(by='nbs')['ga'].as_matrix()
    epi = gadf.query("ensemble=='%s'" %ens).sort_values(by='nbs')['epi'].as_matrix()
    mpl = gadf.query("ensemble=='%s' and nbs==0" %ens)['mpil'].as_matrix()[0]
    awm = hqdf.query("ensemble=='%s'" %ens)['aw0_mean'].as_matrix()[0]
    aws = hqdf.query("ensemble=='%s'" %ens)['aw0_sdev'].as_matrix()[0]
    afs = hqdf.query("ensemble=='%s'" %ens)['alfs'].as_matrix()[0]
    d = gv.dataset.avg_data({'gar': gar, 'epi':epi}, bstrap=True)
    gar_list.append(d['gar'])
    epi_list.append(d['epi'])
    mpl_list.append(mpl)
    aw0_list.append(gv.gvar(awm,aws))
    afs_list.append(afs)
data = {'y':{'gar': gar_list}, 'prior':{'epi': epi_list, 'aw0': aw0_list}, 'x':{'afs': afs_list}, 'mpl': mpl_list}

### Chiral-continuum fit

In [ ]:
# Define fit function

# Setup for Python environment
### Download Anaconda and install 
Download [Anaconda](https://www.continuum.io/downloads) and follow installation instructions.

### Create Python environment with Anaconda
```bash
conda create --name callat_ga python=3 conda
source activate callat_ga
pip install gvar
pip install lsqfit```

Key libraries from [gplepage GitHub](https://github.com/gplepage).
* `gvar` version 8.0
* `lsqfit` version 8.1

Exit conda environment with
```bash
source deactivate
```